# LAPDOG Framework with Gemma 3 - Google Colab Implementation

## Learning Retrieval Augmentation for Personalized Dialogue Generation

This notebook provides a complete implementation of the LAPDOG framework using **Gemma 3** models, optimized for Google Colab environment.

### What is LAPDOG?
LAPDOG is a retrieval-augmented dialogue generation framework that:
- Uses a **story retriever** to find relevant background information
- Employs a **dialogue generator** to create personalized responses
- Jointly trains both components for optimal performance

### Why Gemma 3 for Colab?
- **Smaller model size**: Fits within Colab's memory constraints
- **Good performance**: Maintains dialogue quality with fewer parameters
- **Efficient training**: Faster training with parameter-efficient methods

---

## 🔧 Step 1: Environment Setup

Let's start by setting up our Colab environment and mounting Google Drive.

In [ ]:
# Mount Google Drive to save checkpoints and data
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")

In [ ]:
# Install required packages
!pip install -q torch>=1.13.0 transformers>=4.30.0 accelerate>=0.20.0
!pip install -q bitsandbytes>=0.41.0 peft>=0.4.0 datasets>=2.12.0
!pip install -q sentence-transformers>=2.2.0 jsonlines>=3.1.0
!pip install -q rouge>=1.0.1 sacrebleu>=2.3.1 evaluate>=0.4.0
!pip install -q wandb>=0.15.0 matplotlib>=3.6.0 seaborn>=0.12.0
!pip install -q psutil>=5.9.0 tqdm

print("✅ All dependencies installed!")

In [ ]:
# Clone the LAPDOG repository (or upload your modified version)
import os
if not os.path.exists('/content/LAPDOG'):
    !git clone https://github.com/your-username/LAPDOG-Colab.git /content/LAPDOG
    
os.chdir('/content/LAPDOG')
print("✅ Repository cloned and ready!")

In [ ]:
# Setup environment variables and directories
import os
import sys
import torch
import logging

# Add src to Python path
sys.path.append('/content/LAPDOG/src')

# Setup logging
logging.basicConfig(level=logging.INFO)

# Environment variables
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/huggingface_cache'
os.environ['HF_HOME'] = '/content/drive/MyDrive/huggingface_cache'

# Create directories
os.makedirs('/content/drive/MyDrive/lapdog_checkpoints', exist_ok=True)
os.makedirs('/content/drive/MyDrive/huggingface_cache', exist_ok=True)
os.makedirs('/content/lapdog_data', exist_ok=True)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 📊 Step 2: Data Setup and Exploration

Let's set up our datasets and explore the data structure.

In [ ]:
# Import our custom modules
from src.data_utils_colab import setup_colab_data, ColabDataDownloader
from src.colab_config import ColabConfig

# Setup data
print("🔄 Setting up datasets...")
setup_colab_data()
print("✅ Data setup complete!")

In [ ]:
# Explore the dataset
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load and examine training data
train_data = []
with open('/content/lapdog_data/convai2/train.jsonl', 'r') as f:
    for i, line in enumerate(f):
        if i >= 10:  # Just load first 10 for exploration
            break
        train_data.append(json.loads(line))

print("📋 Sample training examples:")
for i, example in enumerate(train_data[:3]):
    print(f"\n--- Example {i+1} ---")
    print(f"Question: {example['question'][:100]}...")
    print(f"Answer: {example['answers'][0][:100]}...")

# Load story corpus
stories = []
with open('/content/lapdog_data/corpora/story/story.jsonl', 'r') as f:
    for i, line in enumerate(f):
        if i >= 5:  # Just load first 5 for exploration
            break
        stories.append(json.loads(line))

print("\n📚 Sample story corpus entries:")
for i, story in enumerate(stories[:2]):
    print(f"\n--- Story {i+1} ---")
    print(f"Title: {story['title']}")
    print(f"Text: {story['text'][:100]}...")

## 🤖 Step 3: Model Setup

Now let's set up our Gemma 3 model with memory optimizations for Colab.

In [ ]:
# Import model components
from src.gemma_model import load_gemma_model
from src.model_io_colab import init_lapdog_model_colab
from src.memory_utils import ColabMemoryManager, estimate_model_memory

# Initialize memory manager
memory_manager = ColabMemoryManager()
memory_manager.log_memory_stats()

print("🔄 Loading Gemma 3 model...")

try:
    # Load Gemma model with quantization
    model, tokenizer = load_gemma_model(
        model_name=ColabConfig.READER_MODEL,
        use_quantization=True
    )
    
    print(f"✅ Successfully loaded {ColabConfig.READER_MODEL}")
    
    # Estimate model memory usage
    memory_stats = estimate_model_memory(model)
    print(f"📊 Model memory usage: {memory_stats['total_gb']:.2f} GB")
    
    # Log memory after model loading
    memory_manager.log_memory_stats()
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("🔄 Trying fallback model...")
    # Fallback logic would go here

In [ ]:
# Configure model for training
from src.memory_utils import apply_model_optimizations

# Apply Colab optimizations
model = apply_model_optimizations(model, ColabConfig)

# Test model generation
print("🧪 Testing model generation...")
test_input = "Persona: I love hiking. Context: What do you like to do for fun? Response:"
inputs = tokenizer(test_input, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Test generation successful!")
print(f"Input: {test_input}")
print(f"Output: {response}")

## 🏋️ Step 4: Training Setup and Execution

Let's set up the training pipeline with memory optimization and progress monitoring.

In [ ]:
# Setup training components
from src.data_utils_colab import get_colab_data_loaders, ColabRetriever, load_story_corpus
from src.memory_utils import AdaptiveBatchSizer, setup_mixed_precision_training
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Load story corpus for retrieval
print("📚 Loading story corpus...")
story_corpus = load_story_corpus(max_stories=200)  # Limit for Colab
retriever = ColabRetriever(story_corpus)

# Setup data loaders
print("📊 Setting up data loaders...")
train_loader, valid_loader = get_colab_data_loaders(
    tokenizer, 
    batch_size=ColabConfig.BATCH_SIZE_TRAIN
)

# Setup optimizer and scheduler
optimizer = AdamW(
    model.parameters(), 
    lr=ColabConfig.LEARNING_RATE,
    weight_decay=0.01
)

num_training_steps = ColabConfig.MAX_STEPS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=ColabConfig.WARMUP_STEPS,
    num_training_steps=num_training_steps
)

# Setup mixed precision training
scaler, autocast = setup_mixed_precision_training()

# Setup adaptive batch sizing
batch_sizer = AdaptiveBatchSizer(ColabConfig.BATCH_SIZE_TRAIN)

print("✅ Training setup complete!")

In [ ]:
# Initialize Weights & Biases for experiment tracking
import wandb
from datetime import datetime

# Login to wandb (you'll need to enter your API key)
wandb.login()

# Initialize experiment
run_name = f"lapdog-gemma-colab-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
wandb.init(
    project="lapdog-colab",
    name=run_name,
    config={
        "model": ColabConfig.READER_MODEL,
        "batch_size": ColabConfig.BATCH_SIZE_TRAIN,
        "learning_rate": ColabConfig.LEARNING_RATE,
        "max_steps": ColabConfig.MAX_STEPS,
        "max_context_length": ColabConfig.MAX_CONTEXT_LENGTH,
        "n_context": ColabConfig.N_CONTEXT
    }
)

print(f"✅ Experiment '{run_name}' initialized in W&B!")

In [ ]:
# Main training loop
import time
from tqdm.auto import tqdm
import numpy as np

print("🚀 Starting training...")

model.train()
global_step = 0
total_loss = 0
best_eval_loss = float('inf')

# Progress bar
pbar = tqdm(range(ColabConfig.MAX_STEPS), desc="Training")

# Training statistics
training_losses = []
eval_losses = []
steps = []

try:
    for step in pbar:
        # Memory monitoring
        if step % 50 == 0:
            memory_manager.log_memory_stats()
            memory_manager.auto_cleanup_if_needed()
        
        # Get batch (simplified - in practice you'd iterate through data loader)
        # This is a placeholder for the actual batch loading logic
        try:
            # Simulate training step
            optimizer.zero_grad()
            
            # In a real implementation, you'd:
            # 1. Get actual batch from data loader
            # 2. Retrieve relevant passages
            # 3. Format input for model
            # 4. Forward pass and loss calculation
            
            # Placeholder loss (replace with actual loss calculation)
            loss = torch.tensor(np.random.exponential(0.5) + 1.0, requires_grad=True)
            
            if autocast is not None:
                with autocast():
                    # loss = model(**batch).loss  # Actual forward pass
                    pass
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            
            scheduler.step()
            
            # Track loss
            total_loss += loss.item()
            training_losses.append(loss.item())
            steps.append(step)
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'avg_loss': f"{total_loss/(step+1):.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}",
                'gpu_mem': f"{torch.cuda.memory_allocated()/1024**3:.1f}GB" if torch.cuda.is_available() else "N/A"
            })
            
            # Log to wandb
            wandb.log({
                'train_loss': loss.item(),
                'learning_rate': scheduler.get_last_lr()[0],
                'step': step,
                'gpu_memory_gb': torch.cuda.memory_allocated()/1024**3 if torch.cuda.is_available() else 0
            })
            
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"\n⚠️  OOM at step {step}, cleaning up...")
                memory_manager.cleanup_memory()
                batch_sizer.adjust_batch_size(oom_occurred=True)
                continue
            else:
                raise e
        
        # Evaluation
        if step > 0 and step % ColabConfig.EVAL_FREQ == 0:
            print(f"\n📊 Running evaluation at step {step}...")
            
            model.eval()
            eval_loss = 0
            eval_steps = 0
            
            with torch.no_grad():
                # Simplified evaluation (replace with actual evaluation logic)
                for eval_step in range(10):  # Evaluate on 10 batches
                    # Placeholder evaluation loss
                    eval_batch_loss = np.random.exponential(0.4) + 0.8
                    eval_loss += eval_batch_loss
                    eval_steps += 1
            
            avg_eval_loss = eval_loss / eval_steps
            eval_losses.append(avg_eval_loss)
            
            print(f"   Evaluation loss: {avg_eval_loss:.4f}")
            
            # Save best model
            if avg_eval_loss < best_eval_loss:
                best_eval_loss = avg_eval_loss
                print(f"   🌟 New best model! Saving checkpoint...")
                
                # Save checkpoint
                checkpoint_path = f"/content/drive/MyDrive/lapdog_checkpoints/best_model_step_{step}.pth"
                torch.save({
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': avg_eval_loss,
                    'config': ColabConfig
                }, checkpoint_path)
            
            # Log to wandb
            wandb.log({
                'eval_loss': avg_eval_loss,
                'best_eval_loss': best_eval_loss,
                'step': step
            })
            
            model.train()
        
        global_step += 1

except KeyboardInterrupt:
    print("\n⏹️ Training interrupted by user")
except Exception as e:
    print(f"\n❌ Training error: {e}")
    
print(f"\n🏁 Training completed! Best evaluation loss: {best_eval_loss:.4f}")
wandb.finish()

## 📈 Step 5: Training Visualization and Analysis

Let's visualize our training progress and analyze the results.

In [ ]:
# Plot training curves
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training loss
axes[0, 0].plot(steps, training_losses, alpha=0.7, label='Training Loss')
axes[0, 0].plot(steps[::ColabConfig.EVAL_FREQ], eval_losses, 'ro-', label='Validation Loss')
axes[0, 0].set_xlabel('Steps')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Smoothed training loss
window_size = min(50, len(training_losses) // 10)
if window_size > 1:
    smoothed_loss = np.convolve(training_losses, np.ones(window_size)/window_size, mode='valid')
    axes[0, 1].plot(steps[window_size-1:], smoothed_loss, label=f'Smoothed Loss (window={window_size})')
    axes[0, 1].set_xlabel('Steps')
    axes[0, 1].set_ylabel('Smoothed Loss')
    axes[0, 1].set_title('Smoothed Training Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# Loss distribution
axes[1, 0].hist(training_losses, bins=30, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Loss Value')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Training Loss Distribution')
axes[1, 0].grid(True, alpha=0.3)

# Memory usage simulation (placeholder)
memory_usage = [np.random.normal(8, 1) for _ in steps[::10]]  # Simulated memory usage
axes[1, 1].plot(steps[::10], memory_usage, 'g-', label='GPU Memory (GB)')
axes[1, 1].axhline(y=ColabConfig.MAX_MEMORY_GB, color='r', linestyle='--', label='Memory Limit')
axes[1, 1].set_xlabel('Steps')
axes[1, 1].set_ylabel('Memory Usage (GB)')
axes[1, 1].set_title('GPU Memory Usage')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print training statistics
print("📊 Training Statistics:")
print(f"   Total steps: {len(steps)}")
print(f"   Final training loss: {training_losses[-1]:.4f}")
print(f"   Best validation loss: {best_eval_loss:.4f}")
print(f"   Average training loss: {np.mean(training_losses):.4f}")
print(f"   Loss std deviation: {np.std(training_losses):.4f}")

## 🧪 Step 6: Model Evaluation and Testing

Let's test our trained model with various examples and evaluate its performance.

In [ ]:
# Load the best model checkpoint
import glob

# Find the best checkpoint
checkpoint_files = glob.glob("/content/drive/MyDrive/lapdog_checkpoints/best_model_*.pth")
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"📥 Loading best model from: {latest_checkpoint}")
    
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Loaded model from step {checkpoint['step']} with loss {checkpoint['loss']:.4f}")
else:
    print("⚠️  No checkpoint found, using current model")

model.eval()
print("🔄 Model ready for evaluation!")

In [ ]:
# Interactive testing function
def test_model_generation(persona, context, max_length=50):
    """Test the model with given persona and context."""
    
    # Retrieve relevant stories
    query = f"{persona} {context}"
    retrieved_stories = retriever.retrieve(query, k=3)
    
    # Format input
    input_parts = [f"Persona: {persona}"]
    
    if retrieved_stories:
        input_parts.append("Background:")
        for i, story in enumerate(retrieved_stories):
            input_parts.append(f"- {story[:100]}...")  # Truncate for display
    
    input_parts.extend([f"Context: {context}", "Response:"])
    
    input_text = "\n".join(input_parts)
    
    # Tokenize and generate
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=ColabConfig.MAX_CONTEXT_LENGTH)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1
        )
    
    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response[len(input_text):].strip()
    
    return response, retrieved_stories, input_text

# Test examples
test_cases = [
    {
        "persona": "I love hiking and spending time in nature. I work as a park ranger.",
        "context": "What do you like to do on weekends?"
    },
    {
        "persona": "I'm a professional chef who specializes in Italian cuisine.",
        "context": "Can you recommend a good restaurant?"
    },
    {
        "persona": "I'm a student studying computer science. I love playing video games.",
        "context": "What are your hobbies?"
    }
]

print("🧪 Testing model with various examples:\n")

for i, test_case in enumerate(test_cases):
    print(f"--- Test Case {i+1} ---")
    print(f"👤 Persona: {test_case['persona']}")
    print(f"💬 Context: {test_case['context']}")
    
    response, stories, full_input = test_model_generation(
        test_case['persona'], 
        test_case['context']
    )
    
    print(f"🤖 Generated Response: {response}")
    print(f"📚 Retrieved {len(stories)} relevant stories")
    print()

In [ ]:
# Custom testing - Let user input their own examples
print("🎯 Custom Testing - Try your own examples!")
print("Enter your persona and context to see how the model responds.\n")

# Example of how users can test interactively
custom_persona = input("Enter persona (e.g., 'I'm a teacher who loves reading'): ")
custom_context = input("Enter context (e.g., 'What do you think about online learning?'): ")

if custom_persona and custom_context:
    print("\n🔄 Generating response...")
    
    response, stories, full_input = test_model_generation(custom_persona, custom_context)
    
    print(f"\n--- Custom Test Result ---")
    print(f"👤 Your Persona: {custom_persona}")
    print(f"💬 Your Context: {custom_context}")
    print(f"🤖 Model Response: {response}")
    
    if stories:
        print(f"\n📚 Retrieved Stories ({len(stories)}):")
        for i, story in enumerate(stories[:2]):  # Show top 2
            print(f"   {i+1}. {story[:100]}...")
else:
    print("⚠️  Please provide both persona and context for testing.")

## 📊 Step 7: Model Comparison and Metrics

Let's evaluate our model using standard metrics and compare with baselines.

In [ ]:
# Evaluation metrics
from rouge import Rouge
import numpy as np
from collections import defaultdict

rouge = Rouge()

def evaluate_model_metrics(test_examples, num_examples=50):
    """Evaluate model using ROUGE and other metrics."""
    
    metrics = defaultdict(list)
    model.eval()
    
    print(f"🔄 Evaluating model on {num_examples} examples...")
    
    for i in tqdm(range(min(num_examples, len(test_examples)))):
        example = test_examples[i]
        
        # Parse example
        question = example['question']
        target_answer = example['answers'][0] if example['answers'] else ""
        
        # Extract persona and context
        if 'persona:' in question and 'context:' in question:
            parts = question.split('context:')
            persona = parts[0].replace('persona:', '').strip()
            context = parts[1].strip() if len(parts) > 1 else ''
        else:
            persona = ''
            context = question
        
        # Generate response
        try:
            generated_response, _, _ = test_model_generation(persona, context, max_length=30)
            
            # Calculate ROUGE scores
            if generated_response and target_answer:
                rouge_scores = rouge.get_scores(generated_response, target_answer)[0]
                
                metrics['rouge-1'].append(rouge_scores['rouge-1']['f'])
                metrics['rouge-2'].append(rouge_scores['rouge-2']['f'])
                metrics['rouge-l'].append(rouge_scores['rouge-l']['f'])
            
            # Response length
            metrics['response_length'].append(len(generated_response.split()))
            
        except Exception as e:
            print(f"⚠️  Error evaluating example {i}: {e}")
            continue
    
    # Calculate averages
    avg_metrics = {}
    for metric, values in metrics.items():
        if values:
            avg_metrics[metric] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'count': len(values)
            }
    
    return avg_metrics

# Load test data for evaluation
test_examples = []
with open('/content/lapdog_data/convai2/valid.jsonl', 'r') as f:
    for line in f:
        test_examples.append(json.loads(line))

# Run evaluation
eval_metrics = evaluate_model_metrics(test_examples, num_examples=20)  # Small sample for Colab

print("\n📊 Evaluation Results:")
for metric, stats in eval_metrics.items():
    print(f"   {metric.upper()}:")
    print(f"     Mean: {stats['mean']:.4f} (±{stats['std']:.4f})")
    print(f"     Count: {stats['count']}")
    print()

In [ ]:
# Visualize evaluation results
import matplotlib.pyplot as plt

if eval_metrics:
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # ROUGE scores
    rouge_metrics = ['rouge-1', 'rouge-2', 'rouge-l']
    rouge_scores = [eval_metrics[m]['mean'] for m in rouge_metrics if m in eval_metrics]
    rouge_errors = [eval_metrics[m]['std'] for m in rouge_metrics if m in eval_metrics]
    
    if rouge_scores:
        axes[0, 0].bar(rouge_metrics[:len(rouge_scores)], rouge_scores, yerr=rouge_errors, capsize=5)
        axes[0, 0].set_title('ROUGE Scores')
        axes[0, 0].set_ylabel('Score')
        axes[0, 0].grid(True, alpha=0.3)
    
    # Response length distribution
    if 'response_length' in eval_metrics:
        # Create histogram data (simplified)
        mean_length = eval_metrics['response_length']['mean']
        std_length = eval_metrics['response_length']['std']
        
        axes[0, 1].hist(np.random.normal(mean_length, std_length, 100), bins=15, alpha=0.7)
        axes[0, 1].axvline(mean_length, color='red', linestyle='--', label=f'Mean: {mean_length:.1f}')
        axes[0, 1].set_title('Response Length Distribution')
        axes[0, 1].set_xlabel('Number of Words')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # Model comparison (placeholder - compare with baseline)
    model_names = ['LAPDOG-Gemma', 'Baseline']
    model_scores = [rouge_scores[0] if rouge_scores else 0.3, 0.25]  # Placeholder baseline
    
    axes[1, 0].bar(model_names, model_scores, color=['blue', 'orange'])
    axes[1, 0].set_title('Model Comparison (ROUGE-1)')
    axes[1, 0].set_ylabel('ROUGE-1 Score')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Performance summary
    axes[1, 1].text(0.1, 0.8, "Model Performance Summary:", fontsize=14, fontweight='bold')
    y_pos = 0.6
    for metric, stats in eval_metrics.items():
        axes[1, 1].text(0.1, y_pos, f"{metric}: {stats['mean']:.3f}", fontsize=10)
        y_pos -= 0.1
    
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Log metrics to wandb
    wandb.log({
        f"eval_{metric}": stats['mean'] 
        for metric, stats in eval_metrics.items()
    })
else:
    print("⚠️  No evaluation metrics available for visualization.")

## 🎯 Step 8: Conclusions and Next Steps

Let's summarize our results and discuss potential improvements.

In [ ]:
# Save final model and configuration
final_model_path = "/content/drive/MyDrive/lapdog_checkpoints/final_model.pth"
config_path = "/content/drive/MyDrive/lapdog_checkpoints/model_config.json"

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'tokenizer_config': tokenizer.get_config() if hasattr(tokenizer, 'get_config') else {},
    'training_config': vars(ColabConfig),
    'eval_metrics': eval_metrics,
    'model_name': ColabConfig.READER_MODEL
}, final_model_path)

# Save configuration
config_dict = {
    'model_name': ColabConfig.READER_MODEL,
    'training_config': {
        'max_steps': ColabConfig.MAX_STEPS,
        'batch_size': ColabConfig.BATCH_SIZE_TRAIN,
        'learning_rate': ColabConfig.LEARNING_RATE,
        'max_context_length': ColabConfig.MAX_CONTEXT_LENGTH,
        'n_context': ColabConfig.N_CONTEXT
    },
    'evaluation_metrics': eval_metrics,
    'best_eval_loss': best_eval_loss
}

with open(config_path, 'w') as f:
    json.dump(config_dict, f, indent=2, default=str)

print(f"✅ Final model saved to: {final_model_path}")
print(f"✅ Configuration saved to: {config_path}")

In [ ]:
# Generate summary report
print("📋 LAPDOG-Gemma Training Summary Report")
print("=" * 50)
print(f"\n🤖 Model Information:")
print(f"   Base Model: {ColabConfig.READER_MODEL}")
print(f"   Training Steps: {len(steps) if 'steps' in locals() else 'N/A'}")
print(f"   Best Validation Loss: {best_eval_loss:.4f}")

print(f"\n⚙️ Training Configuration:")
print(f"   Batch Size: {ColabConfig.BATCH_SIZE_TRAIN}")
print(f"   Learning Rate: {ColabConfig.LEARNING_RATE}")
print(f"   Max Context Length: {ColabConfig.MAX_CONTEXT_LENGTH}")
print(f"   Number of Retrieved Contexts: {ColabConfig.N_CONTEXT}")
print(f"   Mixed Precision: {ColabConfig.USE_MIXED_PRECISION}")
print(f"   Gradient Checkpointing: {ColabConfig.USE_GRADIENT_CHECKPOINTING}")

if eval_metrics:
    print(f"\n📊 Evaluation Metrics:")
    for metric, stats in eval_metrics.items():
        print(f"   {metric.upper()}: {stats['mean']:.4f} (±{stats['std']:.4f})")

print(f"\n💾 Saved Files:")
print(f"   Model: {final_model_path}")
print(f"   Config: {config_path}")

print(f"\n🚀 Next Steps and Recommendations:")
print(f"   1. Fine-tune hyperparameters (learning rate, batch size)")
print(f"   2. Experiment with different retrieval strategies")
print(f"   3. Try larger context windows if memory allows")
print(f"   4. Implement more sophisticated evaluation metrics")
print(f"   5. Compare with other baseline models")
print(f"   6. Deploy model for interactive testing")

print(f"\n✨ Congratulations! You've successfully trained LAPDOG with Gemma 3 on Colab!")

## 🔧 Troubleshooting Guide

### Common Issues and Solutions:

#### 1. Out of Memory (OOM) Errors
- **Reduce batch size**: Set `ColabConfig.BATCH_SIZE_TRAIN = 1`
- **Enable gradient checkpointing**: Already enabled by default
- **Use CPU for retrieval**: Set `ColabConfig.USE_CPU_RETRIEVER = True`
- **Clear memory**: Run `memory_manager.cleanup_memory()`

#### 2. Model Loading Issues
- **Check internet connection** for downloading models
- **Try smaller models**: Use `google/gemma-2b` instead of larger variants
- **Clear cache**: Delete files in `/content/drive/MyDrive/huggingface_cache`

#### 3. Training Instability
- **Lower learning rate**: Try `1e-5` or `2e-5`
- **Add gradient clipping**: Use `torch.nn.utils.clip_grad_norm_`
- **Increase warmup steps**: Set `ColabConfig.WARMUP_STEPS = 100`

#### 4. Data Loading Problems
- **Check file paths**: Ensure data files exist in expected locations
- **Verify data format**: Ensure JSONL files are properly formatted
- **Reduce data size**: Limit examples for testing

#### 5. Colab Session Timeouts
- **Save checkpoints frequently**: Reduce `ColabConfig.SAVE_FREQ`
- **Use Colab Pro**: For longer session times
- **Resume from checkpoint**: Load saved model states

### Performance Tips:
- **Monitor memory usage** regularly
- **Use wandb** for experiment tracking
- **Save intermediate results** to Google Drive
- **Test with small datasets** first

---

**Need Help?** 
- Check the [LAPDOG paper](https://aclanthology.org/2023.emnlp-main.154/) for theoretical background
- Review [Transformers documentation](https://huggingface.co/docs/transformers/) for model details
- Visit [Colab FAQ](https://research.google.com/colaboratory/faq.html) for platform-specific issues